In [61]:
import sys, json, os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Iterator, Tuple, Optional, List
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import numpy as np
from typing import Tuple
from pathlib import Path
import math

PROJECT_DIR = "/gpfs/Labs/Uzun/SCRIPTS/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER"
SRC_DIR = str(Path(PROJECT_DIR) / "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

from multiomic_transformer.models.model import MultiomicTransformer
from multiomic_transformer.datasets.dataset_refactor import MultiChromosomeDataset, SimpleScaler, fit_simple_scalers
import multiomic_transformer.utils.experiment_loader as experiment_loader



def tanh_scaled_figsize(
    rows: int,
    cols: int,
    short_in: float = 3.0,
    max_ratio: float = 5.0,
    alpha: float = 0.85,
    min_w: float = 2.5,
    max_w: float = 10.0,
    min_h: float = 2.0,
    max_h: float = 10.0,
):
    if rows <= 0 or cols <= 0:
        return (short_in, short_in)

    raw_ratio = max(rows, cols) / max(1, min(rows, cols))  # >= 1
    scaled_ratio = 1.0 + (max_ratio - 1.0) * math.tanh(alpha * math.log(raw_ratio))

    if rows >= cols:
        fig_w = short_in
        fig_h = short_in * scaled_ratio
    else:
        fig_w = short_in * scaled_ratio
        fig_h = short_in

    # ---- NEW: clamp absolute size ----
    fig_w = float(np.clip(fig_w, min_w, max_w))
    fig_h = float(np.clip(fig_h, min_h, max_h))

    return fig_w, fig_h


def downsample_2d_mean(data: np.ndarray, max_rows=1500, max_cols=1500):
    r, c = data.shape
    row_bin = max(1, int(np.ceil(r / max_rows)))
    col_bin = max(1, int(np.ceil(c / max_cols)))

    r2 = (r // row_bin) * row_bin
    c2 = (c // col_bin) * col_bin
    data = data[:r2, :c2]

    data = data.reshape(r2 // row_bin, row_bin, c2 // col_bin, col_bin).mean(axis=(1, 3))
    return data, row_bin, col_bin

def save_input_heatmaps_from_batch(exp, out_dir: Tuple[str, Path], batch_idx: int = 0):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    batch = next(iter(exp.test_loader))
    atac_wins, tf_tensor, tg_expr_true, bias, tf_ids, tg_ids, motif_mask = batch

    # ---- TF expression vector (usually [B, n_tf] or [B, n_tf, 1]) ----
    tf0 = tf_tensor[batch_idx]
    if tf0.ndim == 2 and tf0.shape[-1] == 1:
        tf0 = tf0.squeeze(-1)

    save_heatmap_svg_rasterized_downsampled(
        tf0,
        out_dir / f"input_tf_expr.svg",
        title=f"Input TF Expr",
    )

    # ---- TG true expression (often [B, n_tg] or [B, n_tg, 1]) ----
    tg0 = tg_expr_true[batch_idx]
    if tg0.ndim == 2 and tg0.shape[-1] == 1:
        tg0 = tg0.squeeze(-1)

    save_heatmap_svg_rasterized_downsampled(
        tg0,
        out_dir / f"input_tg_expr_true.svg",
        title=f"Input TG Expr True",
    )
    
    # ---- Batch TG embedding -----
    tg_ids0 = tg_ids.to(exp.model.tg_identity_emb.weight.device)

    tg_id_slice = exp.model.tg_identity_emb.weight[tg_ids0]  # [193, 128]
    save_heatmap_svg_rasterized_downsampled(
        tg_id_slice,
        "./dev/model_heatmaps_svg/tg_identity_emb_slice_batch.svg",
        title="TGs in Batch",
    )

    # ---- Bias (shape varies; make it 2D-ish) ----
    b0 = bias[batch_idx]
    save_heatmap_svg_rasterized_downsampled(
        b0,
        out_dir / f"input_bias.svg",
        title=f"Input Bias",
    )

    # ---- Motif mask (often [B, n_tf, n_tg] or similar) ----
    mm0 = motif_mask[batch_idx]
    save_heatmap_svg_rasterized_downsampled(
        mm0,
        out_dir / f"input_motif_mask.svg",
        title=f"Input Motif Mask",
    )

    # ---- ATAC windows: pick something plottable ----
    # If atac_wins is [B, n_peaks, n_bins] → save as 2D
    # If higher-dim, flatten everything but last dim.
    a0 = atac_wins[batch_idx]
    if a0.ndim > 2:
        a0 = a0.reshape(a0.shape[0], -1)  # keep first dim, flatten rest
    save_heatmap_svg_rasterized_downsampled(
        a0,
        out_dir / f"input_atac_wins.svg",
        title=f"Input ATAC Wins",
        max_rows=1500,
        max_cols=1500,
    )

    print(f"Saved input heatmaps to: {out_dir}")

def save_heatmap_svg_rasterized_downsampled(
    weight_tensor: torch.Tensor,
    out_path: Tuple[str, Path],
    title: str = "",
    cmap: str = "viridis",
    short_in: float = 3.0,
    max_ratio: float = 5.0,
    alpha: float = 0.85,
    dpi: int = 200,
    rasterize: bool = True,
    vmin=None,
    vmax=None,
    max_rows: int = 1500,   # <-- NEW
    max_cols: int = 1500,   # <-- NEW
    is_data: bool = False,
):
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    data = weight_tensor.detach().cpu().numpy()

    if data.ndim == 1:
        data = data[None, :]  # shape (1, N)

    if data.ndim != 2:
        data = np.asarray(data).reshape(data.shape[0], -1)

    rows, cols = data.shape
    if rows > max_rows or cols > max_cols:
        data, row_bin, col_bin = downsample_2d_mean(data, max_rows=max_rows, max_cols=max_cols)
        rows, cols = data.shape
        title = f"{title}"

    fig_w, fig_h = tanh_scaled_figsize(
        rows, cols,
        short_in=short_in,
        max_ratio=max_ratio,
        alpha=alpha,
        min_w=3.0, max_w=9.0,
        min_h=2.0, max_h=7.0,
    )
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    
    if is_data:
        cmap = sns.color_palette("viridis", as_cmap=True)
    else:
        cmap = cmap

    sns.heatmap(
        data,
        cmap=cmap,
        xticklabels=False,
        yticklabels=False,
        cbar=False,
        ax=ax,
        vmin=vmin,
        vmax=vmax,
    )

    if rasterize and ax.collections:
        ax.collections[0].set_rasterized(True)

    title = title.replace("_", " ")
    ax.set_title(title, fontsize=24)
    ax.set_xlabel(f"{cols}", fontsize=32)
    ax.set_ylabel(f"{rows}", fontsize=32)

    fig.tight_layout(pad=0.15)
    fig.savefig(out_path, format=out_path.suffix.lstrip("."), dpi=dpi, bbox_inches="tight", facecolor=fig.get_facecolor())
    plt.close(fig)


def iter_weight_matrices(model: torch.nn.Module) -> Iterator[Tuple[str, torch.Tensor]]:
    """
    Yields (name, weight_tensor) for 2D weight-like tensors.
    Embeddings are treated as 2D weights too.
    """
    for name, param in model.named_parameters():
        if not isinstance(param, torch.Tensor):
            continue
        if param.ndim == 2:
            yield name, param.detach()
        # If you ever want to include 1D params (bias, layernorm), handle separately.


def split_in_proj_weight_qkv(
    in_proj_weight: torch.Tensor,
    d_model: Optional[int] = None,
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Split PyTorch MultiheadAttention in_proj_weight into (Wq, Wk, Wv).
    in_proj_weight shape: (3*d_model, d_model)
    """
    if in_proj_weight.ndim != 2:
        raise ValueError(f"in_proj_weight must be 2D, got {in_proj_weight.shape}")

    rows, cols = in_proj_weight.shape
    if d_model is None:
        d_model = cols

    if rows != 3 * d_model:
        raise ValueError(
            f"Expected in_proj_weight shape (3*d_model, d_model) = ({3*d_model}, {d_model}), "
            f"got {in_proj_weight.shape}"
        )

    Wq = in_proj_weight[0:d_model, :]
    Wk = in_proj_weight[d_model:2*d_model, :]
    Wv = in_proj_weight[2*d_model:3*d_model, :]
    return Wq, Wk, Wv


def save_model_weight_heatmaps(
    model: torch.nn.Module,
    out_dir: Tuple[str, Path],
    include: Optional[List[str]] = None,   # OR logic
    exclude: Optional[List[str]] = None,   # drop if any match
    cmap: str = "viridis",
    max_rows: int = 1500,
    max_cols: int = 1500,
    dpi: int = 200,
    short_in: float = 3.0,
    max_ratio: float = 5.0,
    alpha: float = 0.85,
    split_qkv: bool = True,               # <-- NEW
):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    saved = 0

    for name, w in iter_weight_matrices(model):
        # include/exclude filters
        if include is not None and not any(s in name for s in include):
            continue
        if exclude is not None and any(s in name for s in exclude):
            continue

        # ---- Special case: split encoder self-attn packed QKV ----
        # Typical names:
        #   encoder.layers.0.self_attn.in_proj_weight
        #   encoder.layers.1.self_attn.in_proj_weight
        if split_qkv and name.endswith("self_attn.in_proj_weight"):
            d_model = w.shape[1]  # cols
            Wq, Wk, Wv = split_in_proj_weight_qkv(w, d_model=d_model)

            for tag, mat in [("Q", Wq), ("K", Wk), ("V", Wv)]:
                safe_name = name.replace("/", "_") + f"_{tag}"
                out_path = out_dir / f"{safe_name}.svg"
                save_heatmap_svg_rasterized_downsampled(
                    mat,
                    out_path=out_path,
                    title=f"{name} [{tag}]",
                    cmap=cmap,
                    short_in=short_in,
                    max_ratio=max_ratio,
                    alpha=alpha,
                    dpi=dpi,
                    # max_rows=max_rows,
                    # max_cols=max_cols,
                )
                saved += 1

            continue  # don't save the combined packed matrix unless you also want it
        
        else:
            # ---- Default: save weight as-is ----
            safe_name = name.replace("/", "_")
            out_path = out_dir / f"{safe_name}.svg"
            save_heatmap_svg_rasterized_downsampled(
                w,
                out_path=out_path,
                title=name,
                cmap=cmap,
                short_in=short_in,
                max_ratio=max_ratio,
                alpha=alpha,
                dpi=dpi,
                # max_rows=max_rows,
                # max_cols=max_cols,
            )
        saved += 1

    print(f"Saved {saved} heatmaps to: {out_dir}")

### Load the experiment

In [2]:
exp = experiment_loader.ExperimentLoader(
    experiment_dir = "/gpfs/Labs/Uzun/DATA/PROJECTS/2024.SINGLE_CELL_GRN_INFERENCE.MOELLER/experiments/",
    experiment_name="mESC_E7.5_rep1_hvg_filter_disp_0.5",
    model_num=1,
)

exp.load_trained_model("trained_model.pt")

### Save the Heatmaps of the Model Weights

In [52]:
save_model_weight_heatmaps(
    exp.model,
    "./dev/model_heatmaps_svg",
    split_qkv=True,
)

Saved 41 heatmaps to: dev/model_heatmaps_svg


### Save Heatmaps of the Input Data

In [53]:
save_input_heatmaps_from_batch(exp, "./dev/model_heatmaps_svg", batch_idx=0)

Saved input heatmaps to: dev/model_heatmaps_svg


In [54]:
save_heatmap_svg_rasterized_downsampled(
    exp.model.tf_to_atac_cross_attn_pool.query,
    "./dev/model_heatmaps_svg/tf_to_atac_cross_attn_pool.query.svg",
    title="TF→ATAC Attention Pool Query",
)

save_heatmap_svg_rasterized_downsampled(
    exp.model.atac_to_tf_cross_attn_pool.query,
    "./dev/model_heatmaps_svg/atac_to_tf_cross_attn_pool.query.svg",
    title="ATAC→TF Attention Pool Query",
)

In [73]:
from collections import OrderedDict
import math
import torch

batch = next(iter(exp.test_loader))
atac_wins, tf_tensor, tg_expr_true, bias, tf_ids, tg_ids, motif_mask = batch
device = next(exp.model.parameters()).device

# move tensors you actually compute with
atac_wins  = atac_wins.to(device)
tf_tensor  = tf_tensor.to(device)
tf_ids     = tf_ids.to(device)
tg_ids     = tg_ids.to(device)
bias       = None if bias is None else bias.to(device)
motif_mask = None if motif_mask is None else motif_mask.to(device)

with torch.no_grad():

    # -------------------- ATAC dense input layer --------------------
    atac_lin1        = exp.model.atac_acc_dense_input_layer[0](atac_wins)        # [B, W, d_ff]
    atac_silu        = exp.model.atac_acc_dense_input_layer[1](atac_lin1)        # [B, W, d_ff]
    atac_dropout     = exp.model.atac_acc_dense_input_layer[2](atac_silu)        # [B, W, d_ff]
    atac_lin2        = exp.model.atac_acc_dense_input_layer[3](atac_dropout)     # [B, W, d_model]
    atac_layer_norm  = exp.model.atac_acc_dense_input_layer[4](atac_lin2)        # [B, W, d_model]

    win_emb = atac_layer_norm                                                   # [B, W, d_model]

    # positional encoding + add
    W = win_emb.shape[1]
    pos = torch.arange(W, device=device, dtype=torch.float32)
    pos_emb = exp.model.posenc(pos, bsz=win_emb.shape[0]).transpose(0, 1)        # [B, W, d_model]
    win_emb_pos = win_emb + pos_emb                                             # [B, W, d_model]

    # -------------------- ATAC Transformer encoder --------------------
    window_encoder = exp.model.encoder(win_emb_pos)                               # [B, W, d_model]
    # for consistency with forward(), after this point win_emb refers to encoded windows:
    win_enc = window_encoder                                                     # [B, W, d_model]

    # -------------------- TF embeddings --------------------
    tf_id_emb = exp.model.tf_identity_emb(tf_ids)                                # [T, d_model]

    # TF expr dense input layer (step-by-step)
    tf_expr_in = tf_tensor.unsqueeze(-1)                                         # [B, T, 1]
    tf_lin1    = exp.model.tf_expr_dense_input_layer[0](tf_expr_in)              # [B, T, d_ff]
    tf_silu    = exp.model.tf_expr_dense_input_layer[1](tf_lin1)                 # [B, T, d_ff]
    tf_dropout = exp.model.tf_expr_dense_input_layer[2](tf_silu)                 # [B, T, d_ff]
    tf_lin2    = exp.model.tf_expr_dense_input_layer[3](tf_dropout)              # [B, T, d_model]
    tf_ln      = exp.model.tf_expr_dense_input_layer[4](tf_lin2)                 # [B, T, d_model]
    tf_expr_emb = tf_ln

    # combined TF embedding used for cross-attn
    tf_emb = tf_expr_emb + tf_id_emb.unsqueeze(0)                                # [B, T, d_model]

    # -------------------- TF<->ATAC cross attention --------------------
    # CrossAttention forward: out = norm(query + 0.1*dropout(attn(query, kv)))
    tf_cross   = exp.model.cross_tf_to_atac(tf_emb,  win_enc)                    # [B, T, d_model]
    atac_cross = exp.model.cross_atac_to_tf(win_enc, tf_emb)                     # [B, W, d_model]

    # -------------------- Attention pooling --------------------
    tf_repr,   tf_pool_weights   = exp.model.tf_to_atac_cross_attn_pool(tf_cross)     # [B, d_model], [B, T, 1]
    atac_repr, atac_pool_weights = exp.model.atac_to_tf_cross_attn_pool(atac_cross)  # [B, d_model], [B, W, 1]

    # -------------------- Pooled cross-attn dense layer --------------------
    pooled_cat = torch.cat([tf_repr, atac_repr], dim=-1)                         # [B, 2*d_model]
    pooled_lin1    = exp.model.pooled_cross_attn_dense_layer[0](pooled_cat)      # [B, d_ff]
    pooled_gelu    = exp.model.pooled_cross_attn_dense_layer[1](pooled_lin1)     # [B, d_ff]
    pooled_dropout = exp.model.pooled_cross_attn_dense_layer[2](pooled_gelu)     # [B, d_ff]
    pooled_lin2    = exp.model.pooled_cross_attn_dense_layer[3](pooled_dropout)  # [B, d_model]
    pooled_ln      = exp.model.pooled_cross_attn_dense_layer[4](pooled_lin2)     # [B, d_model]
    tf_atac_cross_attn_output = pooled_ln                                        # [B, d_model]

    # -------------------- TG query / identity embeddings --------------------
    tg_query_emb = exp.model.tg_query_emb(tg_ids)                                # [G, d_model]
    tg_base = tg_query_emb.unsqueeze(0).expand(win_enc.shape[0], -1, -1)         # [B, G, d_model]

    tg_id_emb = exp.model.tg_identity_emb(tg_ids)                                # [G, d_model]

    # -------------------- distance bias shaping (as in forward) --------------------
    attn_bias = None
    if exp.model.use_bias and (bias is not None):
        attn_bias = bias
        if attn_bias.dim() == 3:
            attn_bias = attn_bias.unsqueeze(1)                                   # [B, 1, G, W]
        if attn_bias.shape[1] == 1:
            attn_bias = attn_bias.expand(win_enc.shape[0], exp.model.num_heads, tg_base.size(1), win_enc.size(1))
        attn_bias = torch.nan_to_num(attn_bias, nan=0.0, posinf=1e4, neginf=-1e4)
        attn_bias = (exp.model.bias_scale * attn_bias).clamp_(-20.0, 20.0)       # [B, H, G, W]

    # -------------------- TG->ATAC cross attention --------------------
    tg_cross = exp.model.cross_tg_to_atac(tg_base, win_enc, attn_bias=attn_bias) # [B, G, d_model]

    # add pooled TF/ATAC summary to each TG (scaled)
    n_tgs = tg_cross.size(1)
    scale = 1.0 / math.sqrt(max(1, n_tgs))
    tf_atac_expand = tf_atac_cross_attn_output.unsqueeze(1).expand(-1, n_tgs, -1) * scale  # [B, G, d_model]
    tg_cross_attn_repr = tg_cross + tf_atac_expand                                # [B, G, d_model]

    # -------------------- TG identity dot + gene_pred_dense --------------------
    tg_similarity_to_attn_output = (tg_cross_attn_repr * tg_id_emb.unsqueeze(0)).sum(dim=-1)  # [B, G]

    gene_lin1    = exp.model.gene_pred_dense[0](tg_cross_attn_repr)               # [B, G, d_ff]
    gene_relu    = exp.model.gene_pred_dense[1](gene_lin1)                        # [B, G, d_ff]
    gene_dropout = exp.model.gene_pred_dense[2](gene_relu)                        # [B, G, d_ff]
    gene_lin2    = exp.model.gene_pred_dense[3](gene_dropout)                     # [B, G, 1]
    gene_pred_term = gene_lin2.squeeze(-1)                                        # [B, G]

    tg_pred_pre_shortcut = tg_similarity_to_attn_output + gene_pred_term          # [B, G]

    # -------------------- Optional TF->TG shortcut --------------------
    shortcut_out = None
    shortcut_attn = None
    tg_pred = tg_pred_pre_shortcut

    if getattr(exp.model, "use_shortcut", False) and hasattr(exp.model, "shortcut_layer"):
        # shortcut_layer expects: tg_emb [G,d], tf_id_emb [T,d], tf_expr [B,T], motif_mask [G,T]
        # If your motif_mask is batched, slice batch 0; otherwise pass as-is.
        mm_for_shortcut = None
        if motif_mask is not None:
            mm_for_shortcut = motif_mask[0] if motif_mask.dim() == 3 else motif_mask

        shortcut_out, shortcut_attn = exp.model.shortcut_layer(
            tg_id_emb, tf_id_emb, tf_tensor, motif_mask=mm_for_shortcut
        )  # shortcut_out: [B,G], shortcut_attn: [G,T]
        tg_pred = tg_pred + shortcut_out                                          # [B, G]


# -------------------- model_dict (grouped outputs) --------------------
model_dict = OrderedDict({
    "inputs": [
        ("atac_windows", atac_wins),
        ("tf_expr", tf_tensor),
        ("tg_expr_true", tg_expr_true),
        ("bias", bias),
        ("tf_ids", tf_ids),
        ("tg_ids", tg_ids),
        ("motif_mask", motif_mask),
    ],

    "atac_dense_input_layer": [
        ("lin1", atac_lin1),
        ("silu", atac_silu),
        ("dropout", atac_dropout),
        ("lin2", atac_lin2),
        ("layer_norm", atac_layer_norm),
        ("pos_emb", pos_emb),
        ("win_emb_pos", win_emb_pos),
    ],

    "window_encoder": [
        ("output", window_encoder),
    ],

    "tf_id_emb": [
        ("embedding", tf_id_emb),
    ],

    "tf_dense_input_layer": [
        ("expr_in", tf_expr_in),
        ("lin1", tf_lin1),
        ("silu", tf_silu),
        ("dropout", tf_dropout),
        ("lin2", tf_lin2),
        ("layer_norm", tf_ln),
    ],

    "tf_emb_combined": [
        ("tf_expr_emb", tf_expr_emb),
        ("tf_emb", tf_emb),
    ],

    "cross_tf_to_atac": [
        ("tf_cross", tf_cross),
    ],

    "cross_atac_to_tf": [
        ("atac_cross", atac_cross),
    ],

    "attention_pooling": [
        ("tf_repr", tf_repr),
        ("tf_pool_weights", tf_pool_weights),
        ("atac_repr", atac_repr),
        ("atac_pool_weights", atac_pool_weights),
    ],

    "pooled_cross_attn_dense_layer": [
        ("cat_tf_atac", pooled_cat),
        ("lin1", pooled_lin1),
        ("gelu", pooled_gelu),
        ("dropout", pooled_dropout),
        ("lin2", pooled_lin2),
        ("layer_norm", pooled_ln),
        ("tf_atac_cross_attn_output", tf_atac_cross_attn_output),
    ],

    "tg_embeddings": [
        ("tg_query_emb", tg_query_emb),
        ("tg_base", tg_base),
        ("tg_id_emb", tg_id_emb),
    ],

    "tg_to_atac_bias": [
        ("attn_bias", attn_bias),
    ],

    "cross_tg_to_atac": [
        ("tg_cross", tg_cross),
    ],

    "tg_cross_attn_fusion": [
        ("tf_atac_expand", tf_atac_expand),
        ("tg_cross_attn_repr", tg_cross_attn_repr),
    ],

    "tg_prediction_head": [
        ("tg_similarity_to_attn_output", tg_similarity_to_attn_output),
        ("gene_lin1", gene_lin1),
        ("gene_relu", gene_relu),
        ("gene_dropout", gene_dropout),
        ("gene_lin2", gene_lin2),
        ("gene_pred_term", gene_pred_term),
        ("tg_pred_pre_shortcut", tg_pred_pre_shortcut),
        ("tg_pred_final", tg_pred),
    ],

    "tf_tg_shortcut": [
        ("shortcut_out", shortcut_out),
        ("shortcut_attn", shortcut_attn),
    ],
})


In [79]:
import re
import torch

def _slug(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)
    return s.strip("_")

def _select_sample(x: torch.Tensor, batch_idx: int = 0):
    """
    Make x plottable as 1D/2D:
    - If x is [B, ...], take x[batch_idx]
    - If x is >2D after that, flatten to 2D (keep first dim)
    - If x is 1D, keep 1D (your heatmap fn will expand to (1,N))
    """
    if isinstance(x, torch.Tensor) and x.ndim >= 3:
        x = x[batch_idx]

    # after slicing, flatten >2D to 2D
    if isinstance(x, torch.Tensor) and x.ndim > 2:
        x = x.reshape(x.shape[0], -1)

    return x

def save_grouped_heatmaps(
    model_dict,
    out_root: Tuple[str, Path],
    batch_idx: int = 0,
    dpi: int = 200,
    cmap: str = "viridis",
    skip_none: bool = True,
):
    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    # number of digits for folder numbering
    n_parts = len(model_dict)
    part_digits = max(2, len(str(n_parts - 1)))

    for part_i, (part_name, items) in enumerate(model_dict.items()):

        # ---- Numbered subdirectory ----
        part_slug = _slug(part_name)
        part_dir = out_root / f"{part_i:0{part_digits}d}_{part_slug}"
        part_dir.mkdir(parents=True, exist_ok=True)

        # digits for files inside this part
        n_items = len(items)
        file_digits = max(2, len(str(n_items - 1)))

        saved_i = 0
        for item_name, tensor in items:

            # ---- Skip None or non-tensors ----
            if tensor is None:
                if skip_none:
                    continue
                else:
                    raise ValueError(f"{part_name}/{item_name} is None")

            if not torch.is_tensor(tensor):
                if skip_none:
                    continue
                else:
                    raise TypeError(f"{part_name}/{item_name} is not a tensor: {type(tensor)}")

            x = _select_sample(tensor, batch_idx=batch_idx)

            # ---- Numbered file ----
            fname = f"{saved_i:0{file_digits}d}_{_slug(item_name)}.svg"
            out_path = part_dir / fname

            # ---- Two-line title ----
            title = f"{part_name}\n{item_name}"

            save_heatmap_svg_rasterized_downsampled(
                x,
                out_path,
                title=title,
                cmap=cmap,
                dpi=dpi,
            )

            saved_i += 1

    print(f"Saved grouped heatmaps to: {out_root}")



save_grouped_heatmaps(
    model_dict,
    out_root="./dev/model_heatmaps_svg",
    batch_idx=0,
    dpi=200,
)


Saved grouped heatmaps to: dev/model_heatmaps_svg


In [84]:
import re
from pathlib import Path
import networkx as nx
import torch

def _slug(s: str) -> str:
    s = s.strip().replace(" ", "_")
    s = re.sub(r"[^\w\-]+", "_", s)
    s = re.sub(r"_+", "_", s)
    return s.strip("_")

def subdir_name(part_i: int, part_name: str, part_digits: int = 2) -> str:
    return f"{part_i:0{part_digits}d}_{_slug(part_name)}"

def file_name(item_i: int, item_name: str, file_digits: int = 2, ext: str = ".svg") -> str:
    return f"{item_i:0{file_digits}d}_{_slug(item_name)}{ext}"

def data_node_path(out_root: Path, part_i: int, part_name: str, item_i: int, item_name: str) -> str:
    sd = subdir_name(part_i, part_name)
    fn = file_name(item_i, item_name)
    return str((out_root / sd / fn).as_posix())

def data_node_id(part_i: int, part_name: str, item_i: int, item_name: str) -> str:
    # what you want to SEE in the graph
    sd = subdir_name(part_i, part_name)
    fn = file_name(item_i, item_name, ext="")  # no .svg in label
    return f"{sd}/{fn}"

def weight_node_id(weights_subdir: str, param_name: str, tag: Optional[str] = None) -> str:
    base = f"{weights_subdir}/{param_name.replace('/', '_')}"
    return f"{base}_{tag}" if tag else base


def weight_node_paths(out_root: Path, weights_subdir: str, param_name: str) -> list[str]:
    """
    Returns list of svg paths for the given param, mirroring save_model_weight_heatmaps().
    """
    base_dir = out_root / weights_subdir
    base_dir.mkdir(parents=True, exist_ok=True)

    if param_name.endswith("self_attn.in_proj_weight"):
        # split QKV
        safe = param_name.replace("/", "_")
        return [
            str((base_dir / f"{safe}_Q.svg").as_posix()),
            str((base_dir / f"{safe}_K.svg").as_posix()),
            str((base_dir / f"{safe}_V.svg").as_posix()),
        ]
    else:
        safe = param_name.replace("/", "_")
        return [str((base_dir / f"{safe}.svg").as_posix())]

def build_data_index(model_dict, out_root: Path) -> dict[tuple[str, str], dict]:
    """
    Map (part_name, item_name) -> {"id": node_id, "path": svg_path}
    """
    idx = {}
    parts = list(model_dict.items())
    part_digits = max(2, len(str(len(parts) - 1)))

    for part_i, (part_name, items) in enumerate(parts):
        file_digits = max(2, len(str(len(items) - 1)))

        saved_i = 0
        for item_name, tensor in items:
            if tensor is None or (not torch.is_tensor(tensor)):
                continue

            sd = subdir_name(part_i, part_name, part_digits=part_digits)
            fn = file_name(saved_i, item_name, file_digits=file_digits, ext=".svg")

            node_id = f"{sd}/{Path(fn).stem}"                # << graph label
            svg_path = str((out_root / sd / fn).as_posix())  # << actual file

            idx[(part_name, item_name)] = {"id": node_id, "path": svg_path}
            saved_i += 1

    return idx



def build_param_set(model: torch.nn.Module) -> set[str]:
    return {name for name, _ in model.named_parameters()}


def add_weight_nodes_for_param(G: nx.DiGraph, out_root: Path, weights_subdir: str, param_name: str):
    base_dir = out_root / weights_subdir
    safe = param_name.replace("/", "_")

    if param_name.endswith("self_attn.in_proj_weight"):
        for tag in ["Q", "K", "V"]:
            node_id = weight_node_id(weights_subdir, param_name, tag=tag)
            svg_path = str((base_dir / f"{safe}_{tag}.svg").as_posix())
            if not G.has_node(node_id):
                G.add_node(node_id, kind="weight", param=param_name, path=svg_path, tag=tag)
    else:
        node_id = weight_node_id(weights_subdir, param_name, tag=None)
        svg_path = str((base_dir / f"{safe}.svg").as_posix())
        if not G.has_node(node_id):
            G.add_node(node_id, kind="weight", param=param_name, path=svg_path)



def build_flow_graph(
    model_dict,
    model: torch.nn.Module,
    out_root: Tuple[str, Path] = "./dev/model_heatmaps_svg",
    weights_subdir: str = "00_weights",  # numbered so it sorts too
) -> nx.DiGraph:
    out_root = Path(out_root)

    G = nx.DiGraph()

    # 1) add data nodes from model_dict
    data_idx = build_data_index(model_dict, out_root)
    for (part_name, item_name), meta in data_idx.items():
        G.add_node(
            meta["id"],
            kind="data",
            part=part_name,
            item=item_name,
            path=meta["path"],
        )

    # 2) add weight nodes from model parameters (only when referenced)
    param_names = build_param_set(model)

    def link(data_src_key, param_name, data_dst_key):
        if data_src_key not in data_idx or data_dst_key not in data_idx:
            return
        if param_name not in param_names:
            return

        src_id = data_idx[data_src_key]["id"]
        dst_id = data_idx[data_dst_key]["id"]

        # ensure data nodes exist (with path attr)
        if not G.has_node(src_id):
            G.add_node(src_id, kind="data", **data_idx[data_src_key])
        if not G.has_node(dst_id):
            G.add_node(dst_id, kind="data", **data_idx[data_dst_key])

        add_weight_nodes_for_param(G, out_root, weights_subdir, param_name)

        # connect through weight nodes
        if param_name.endswith("self_attn.in_proj_weight"):
            for tag in ["Q", "K", "V"]:
                w_id = weight_node_id(weights_subdir, param_name, tag=tag)
                G.add_edge(src_id, w_id, kind="uses")
                G.add_edge(w_id, dst_id, kind="produces")
        else:
            w_id = weight_node_id(weights_subdir, param_name, tag=None)
            G.add_edge(src_id, w_id, kind="uses")
            G.add_edge(w_id, dst_id, kind="produces")


    # 3) Wiring spec: (src data) --param--> (dst data)

    # ---- ATAC dense input layer ----
    link(("atac_dense_input_layer", "atac_windows"), "atac_acc_dense_input_layer.0.weight", ("atac_dense_input_layer", "lin1"))
    link(("atac_dense_input_layer", "atac_windows"), "atac_acc_dense_input_layer.0.bias",   ("atac_dense_input_layer", "lin1"))
    link(("atac_dense_input_layer", "lin1"),        "atac_acc_dense_input_layer.3.weight", ("atac_dense_input_layer", "lin2"))
    # LN has weight/bias
    link(("atac_dense_input_layer", "lin2"),        "atac_acc_dense_input_layer.4.weight", ("atac_dense_input_layer", "layer_norm"))
    link(("atac_dense_input_layer", "lin2"),        "atac_acc_dense_input_layer.4.bias",   ("atac_dense_input_layer", "layer_norm"))
    # pos add is not a learned weight matrix here, but we can connect data->data directly:
    if ("atac_dense_input_layer", "layer_norm") in data_idx and ("atac_dense_input_layer", "win_emb_pos") in data_idx:
        src_id = data_idx[("atac_dense_input_layer", "layer_norm")]["id"]
        dst_id = data_idx[("atac_dense_input_layer", "win_emb_pos")]["id"]
        G.add_edge(src_id, dst_id, kind="add_posenc")
    # ---- Encoder layers (0-2) ----
    for L in range(3):
        # in_proj_weight splits into Q/K/V files
        link(("atac_dense_input_layer", "win_emb_pos"),
             f"encoder.layers.{L}.self_attn.in_proj_weight",
             ("window_encoder", "output"))
        link(("atac_dense_input_layer", "win_emb_pos"),
             f"encoder.layers.{L}.self_attn.out_proj.weight",
             ("window_encoder", "output"))
        link(("atac_dense_input_layer", "win_emb_pos"),
             f"encoder.layers.{L}.linear1.weight",
             ("window_encoder", "output"))
        link(("atac_dense_input_layer", "win_emb_pos"),
             f"encoder.layers.{L}.linear2.weight",
             ("window_encoder", "output"))

    # ---- TF dense input layer ----
    link(("tf_dense_input_layer", "expr_in"), "tf_expr_dense_input_layer.0.weight", ("tf_dense_input_layer", "lin1"))
    link(("tf_dense_input_layer", "expr_in"), "tf_expr_dense_input_layer.0.bias",   ("tf_dense_input_layer", "lin1"))
    link(("tf_dense_input_layer", "lin2"),    "tf_expr_dense_input_layer.4.weight", ("tf_dense_input_layer", "layer_norm"))
    link(("tf_dense_input_layer", "lin2"),    "tf_expr_dense_input_layer.4.bias",   ("tf_dense_input_layer", "layer_norm"))

    # ---- TF id embedding table (treated as weight) ----
    # Connect ids -> embedding output via embedding weight
    link(("inputs", "tf_ids"), "tf_identity_emb.weight", ("tf_id_emb", "embedding"))

    # ---- Cross TF->ATAC (use Wq/Wk/Wv/Wo) ----
    for p in ["W_q.weight","W_k.weight","W_v.weight","W_o.weight"]:
        link(("tf_emb_combined", "tf_emb"), f"cross_tf_to_atac.attn.{p}", ("cross_tf_to_atac", "tf_cross"))

    # ---- Cross ATAC->TF ----
    for p in ["W_q.weight","W_k.weight","W_v.weight","W_o.weight"]:
        link(("window_encoder", "output"), f"cross_atac_to_tf.attn.{p}", ("cross_atac_to_tf", "atac_cross"))

    # ---- Attention pooling queries ----
    link(("cross_tf_to_atac", "tf_cross"), "tf_to_atac_cross_attn_pool.query", ("attention_pooling", "tf_repr"))
    link(("cross_atac_to_tf", "atac_cross"), "atac_to_tf_cross_attn_pool.query", ("attention_pooling", "atac_repr"))

    # ---- Pooled dense ----
    link(("pooled_cross_attn_dense_layer", "cat_tf_atac"), "pooled_cross_attn_dense_layer.0.weight", ("pooled_cross_attn_dense_layer", "lin1"))
    link(("pooled_cross_attn_dense_layer", "lin2"),        "pooled_cross_attn_dense_layer.4.weight", ("pooled_cross_attn_dense_layer", "layer_norm"))
    link(("pooled_cross_attn_dense_layer", "lin2"),        "pooled_cross_attn_dense_layer.4.bias",   ("pooled_cross_attn_dense_layer", "layer_norm"))

    # ---- TG embeddings ----
    link(("inputs", "tg_ids"), "tg_query_emb.weight",    ("tg_embeddings", "tg_query_emb"))
    link(("inputs", "tg_ids"), "tg_identity_emb.weight", ("tg_embeddings", "tg_id_emb"))

    # ---- Cross TG->ATAC ----
    for p in ["W_q.weight","W_k.weight","W_v.weight","W_o.weight"]:
        link(("tg_embeddings", "tg_base"), f"cross_tg_to_atac.attn.{p}", ("cross_tg_to_atac", "tg_cross"))

    # ---- Gene pred dense ----
    link(("tg_cross_attn_fusion", "tg_cross_attn_repr"), "gene_pred_dense.0.weight", ("tg_prediction_head", "gene_lin1"))
    link(("tg_prediction_head", "gene_dropout"),         "gene_pred_dense.3.weight", ("tg_prediction_head", "gene_lin2"))

    return G

G = build_flow_graph(model_dict, exp.model, out_root="./dev/model_heatmaps_svg", weights_subdir="00_weights")
nx.write_graphml(G, "./dev/model_heatmaps_svg/model_flow.graphml")
print("Wrote graph:", "./dev/model_heatmaps_svg/model_flow.graphml")

save_model_weight_heatmaps(
    exp.model,
    "./dev/model_heatmaps_svg/00_weights",
    split_qkv=True,
)


Wrote graph: ./dev/model_heatmaps_svg/model_flow.graphml
Saved 41 heatmaps to: dev/model_heatmaps_svg/00_weights
